# Functions

We'll save our functions in this file

In [ ]:
def dataframe():
    
    df = pd.read("ftse_base4.csv", parse_dates=["time"])
    
    return df


In [ ]:
#Create the flexible outcome feature

#copy the exisiting df in to a new dataframe to make the function easily recyclable
df1 = df

#Set a variable that we can increment from within the function
x = 0 

def outcome_generator(row, target, drawdown):
   
    global x #let's make our variable global so we can change it's value from inside this function
    
    
    #We'll add a print report to show the progress we're making mid function execute
    if x%200 == 0: 
        print("processing row {}".format(x))
    
    
    #create variables for each of the rows we may need to iterate over
    high = df["high"]
    low = df["low"]
    close = df["close"]
    open_ = df["open"]
    signal = df["signal"]
    direction = df["direction"]
    
   
    
    long_short = "" #set an empty variable to take a long or short position 
    
#     if x == 0: 
#         x += 1
#         return "unknown" #We can't know what happens at row 0 as there's no earlier data to inform our decision. 
    
    
    #let's first check what the trade signal is and store it in our local long_short variable. 
    if signal[x] == "no_trade":
        x += 1
        return "no_trade"
    elif signal[x] == "unknown":
        x += 1
        return "no_trade"
    elif signal[x] == "trade" and direction[x] == "long":
        long_short = "long"
    elif signal[x] == "trade" and direction[x] == "short":
        long_short = "short"
    
        
    #Set a counter for the upcoming loop, we want to be able to count out three candles
    short_count = 0 
    long_count = 0 
    
    can_list = [] #create an empty container to store an outcome for the three candles that determine trade success 

    '''
    First we'll focus on the short trade signals. We'll rehash the code use in trade_class
    '''
    
    #Now we find out if a short trade is successful or not based on next 3 candle values
    for i, (j, k, m, n) in enumerate(zip(high[x+1:], low[x+1:], close[x+1:], open_[x+1:])): 
        
        if len(can_list) < 3 and long_short == "short": #check both conditions are true before proceeding. 
            if j > close[x]+drawdown: #We check if the high of the next 30minute candle is higher than our signal candle high

                can_list.append('loss') #add sub_outcome 'loss' to our value_list container. 
                    
            elif close[x] - k > target: #Determines whether we've hit the target

                can_list.append('win')
                
            elif close[x] < m: #Determines whether a reversal signal has been printed
                
                can_list.append('loss')
                
            else:

                can_list.append('no_score') #applicable for a minor retrace or small winning position < 10
        
        elif len(can_list) < 3 and long_short == "long":
            if k < close[x]-drawdown: #We check out if the low of the next 30minute candle is lower than our signal candle

                can_list.append('loss')#if so then add this to the temp_list
                    
            elif j - close[x] > target: #Determine whether we've hit the target

                can_list.append('win')
            
            elif m < close[x]:
                
                can_list.append('loss')
                    
            else:

                can_list.append('no_score')
           
    print(x, can_list)
    '''
    We now look at the can_list elements and check to see what comes first - win or loss. And return whichever of the 
    two outcomes is first. If a win or loss is not recorded we return no score. 
    '''
    if len(can_list)==3: #If our container has stored the next 3 candle outcomes, proceed. 
        if can_list[0] == 'loss':
            x+=1 #increment x by one
            return 'loss' #and return the trade outcome
        elif can_list[0] == 'win':
            x+=1
            return 'win'
        elif can_list[0] == 'no_score' and can_list[1] == 'loss':
            x+=1
            return 'loss'
        elif can_list[0] == 'no_score' and can_list[1] == 'win':
            x+=1
            return 'win'
        elif can_list[0] == 'no_score' and can_list[1] == 'no_score' and can_list[2] == 'loss':
            x+=1
            return 'loss'
        elif can_list[0] == 'no_score' and can_list[1] == 'no_score' and can_list[2] == 'win':
            x+=1
            return 'win'
        else:
            x+=1
            return 'no_score'

In [ ]:
#Create profit feature using multiple arguments for flexible parameters

df2 = df1

x = 0  #our current row loc stored in a global variable

def profit_calc(row, target, drawdown):
    """
    In this function we will calculate the result of each trade in terms of pips gained or lost. 
    
    """
    global x #let's make our variable global so we can change it's value from inside this function
    
    
    #We'll add a print report to show the progress we're making mid function execute
    if x%250 == 0: 
        print("processing row {}".format(x))
    
    
    #create variables for each of the rows we may need to iterate over
    high = df2['high'] 
    low = df2['low'] 
    close = df2['close']
    open_ = df2['open']
    outcome = df2['outcome']
    direction = df2['direction']
    
   
    
    long_short = "" #set an empty variable to take a long or short position 
    win_loss = "" #set an empty variable to take a win or loss outcome
    
    if x == 0: 
        x += 1
        return 0 #We can't know what happens at row 0 as there's no earlier data to inform our decision. 
    
    
    #let's first check what the trade signal is and store it to our local long_short variable
    if direction[x] == direction[x-1]:
        x += 1
        return 0
    elif direction[x] == "neutral":
        x += 1
        return 0
    elif direction[x] == "short":
        long_short = "short"
    else:
        long_short = "long"
    
    #Check what the outcome was and store this to our local win_loss variable
    if outcome[x] == "no_score":
        x+=1
        return 0
    elif outcome[x] == "win":
        win_loss = "win"
    elif outcome[x] == "loss":
        win_loss = "loss"
    else:
        x+=1
        return 0
        
    #Set counters so we can keep a track of how many candles we've valued    
    short_count = 0 
    long_count = 0 
    
    can_list = [0] #create an empty container to store our trade sub-outcomes
    

   
    #first we'll calculate losses. 
    #A loss is printed when we're stopped out or when a reversal signal is activated. 
    for i, (j, k, m, n, o) in enumerate(zip(high[x+1:], low[x+1:], close[x+1:], open_[x+1:], direction[x+1:])):
        if win_loss == "loss" and long_short == "short":
            #stopped out
            if j > close[x]+drawdown:
                result = - drawdown
                x+=1
                
                return result
            
            #reversal signal so only when the canlde closes in opposite direction to our candle
            elif j < (close[x]+drawdown) and o != direction[x]:
                result = close[x] - m
                x+=1
                
                return result
        
        elif win_loss == "loss" and long_short == "long":
            #stopped out
            if k < close[x]-drawdown:
                result = - drawdown
                x+=1
                
                return result
            
            #reversal signal so only applicable for candles closing in opposite direction to signal
            elif k > (close[x]-drawdown) and o != direction[x]:
                result = m - close[x]
                x+=1
                
                return result
            
    
    #Now we'll calculate wins based on the values of five subsequent candles
    for i, (j, k, m, n, o) in enumerate(zip(high[x+1:], low[x+1:], close[x+1:], open_[x+1:], direction[x+1:])):
        #Check we have less than five entries in our can_list and check that we haven't seen a reversal signal?
        if o == direction[x] and len(can_list) < 6:
            
            
            #our short win is the distance from the signal close to the low of the subsequent 5 candles
            if win_loss == "win" and long_short == "short":
                
                can_list.append(abs(close[x] - k))
            
            #our long win is the distance from the signal close to the high of the subsequent 5 candles
            elif win_loss == "win" and long_short == "long":
                
                can_list.append(abs(j - close[x]))
                
        elif o != direction[x]:
            break
    
    #print the can_list to check our win loop/conditional is performing as expected?
    print(x, can_list, len(can_list))

    
    x+=1    
    
    
    
    #If our can_list has not been populated, we'll just return our target, else we'll return the max value. 
    if len(can_list) < 2:
        return target 
    else:
        return max(can_list)